In [21]:
!pip install numpy==1.20.0

In [22]:
!pip install mediapipe

In [23]:
!pip install wget

In [24]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [25]:
model_source = 'https://storage.googleapis.com/mediapipe-models/object_detector/efficientdet_lite0/int8/latest/efficientdet_lite0.tflite'
image_source = 'https://storage.googleapis.com/mediapipe-tasks/object_detector/cat_and_dog.jpg'

asserts_path = "./asserts/"

image_filename = 'image.jpg'
model_filename = 'efficientdet.tflite'

image_path = asserts_path + image_filename
model_path = asserts_path + model_filename

In [26]:
import wget
wget.download(image_source,image_path)
wget.download(model_source,model_path)

'./asserts/efficientdet (1).tflite'

In [27]:
import cv2

img = cv2.imread(image_path)
cv2.imshow('image',img)
cv2.waitKey(0)

-1

In [28]:
#@markdown We implemented some functions to visualize the object detection results. <br/> Run the following cell to activate the functions.
import cv2
import numpy as np

MARGIN = 10  # pixels
ROW_SIZE = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
TEXT_COLOR = (255, 0, 0)  # red


def visualize(
    image,
    detection_result
) -> np.ndarray:
  """Draws bounding boxes on the input image and return it.
  Args:
    image: The input RGB image.
    detection_result: The list of all "Detection" entities to be visualize.
  Returns:
    Image with bounding boxes.
  """
  for detection in detection_result.detections:
    # Draw bounding_box
    bbox = detection.bounding_box
    start_point = bbox.origin_x, bbox.origin_y
    end_point = bbox.origin_x + bbox.width, bbox.origin_y + bbox.height
    cv2.rectangle(image, start_point, end_point, TEXT_COLOR, 3)

    # Draw label and score
    category = detection.categories[0]
    category_name = category.category_name
    probability = round(category.score, 2)
    result_text = category_name + ' (' + str(probability) + ')'
    text_location = (MARGIN + bbox.origin_x,
                     MARGIN + ROW_SIZE + bbox.origin_y)
    cv2.putText(image, result_text, text_location, cv2.FONT_HERSHEY_PLAIN,
                FONT_SIZE, TEXT_COLOR, FONT_THICKNESS)

  return image

In [30]:
import mediapipe as mp
import numpy as np

BaseOptions = mp.tasks.BaseOptions
ObjectDetector = mp.tasks.vision.ObjectDetector
ObjectDetectorOptions = mp.tasks.vision.ObjectDetectorOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = ObjectDetectorOptions(
    base_options=BaseOptions(model_asset_path = model_path),
    max_results=5,
    score_threshold=0.5,
    running_mode=VisionRunningMode.IMAGE)

with ObjectDetector.create_from_options(options) as detector:
    image = mp.Image.create_from_file(image_path)
    detection_result = detector.detect(image)

image_copy = np.copy(image.numpy_view())
annotated_image = visualize(image_copy, detection_result)
rgb_annotated_image = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
cv2.imshow('Detection Result',rgb_annotated_image)
cv2.waitKey(0)

-1